## Сергей Гришаев
## Architect @ SberMarket

In [ ]:
from IPython.display import IFrame, Image

## Мотивация создания Apache Spark

### Рассмотрим два примера приложений:
- Обучить модель на больших данных (итеративный алгоритм над фиксированным датасетом)
- Провести ad-hoc анализ данных из двух таблиц (выполнить несколько интерактивных запросов с джойнами и группировками)

## Основные недостатки классического MapReduce

- Постоянное чтение/запись во внешнее хранилище
- Сложный API
- Ограниченное число источников/приемников данных
- MapReduce - это только вычислительный фреймворк

https://arxiv.org/pdf/1209.2191.pdf MapReduce is Good Enough? If All You Have is a Hammer, Throw Away Everything That’s Not a Nail! 

<img src="pics/mapper_2.png" width=700/>

<img src="pics/reducer.png" width=700/>

## Apache Spark - это *быстрая* распределенная вычислительная платформа *общего назначения*

1. **Быстрая** 

    - Кеширование рабочих данных в памяти
    - Ленивые вычисления


2. **Общего назначения** 

    - Можно реализовать любые вычисления (батчевые, итеративные, интерактивные, в режиме реального времени)
    - Более общая модель программирования (узлы в графе вычислений могут быть не только map и reduce)
    - Высокоуровневый API (Scala, Java, Python, R)
    - Локальная установка



<img src="pics/spark_stack.png" width=1000/>

## Множество источников данных

<img src="pics/spark_data_sources.jpg" width=1000/>

## Архитектура Apache Spark

<img src="pics/SparkYARN.png" width=800/>

## Запуск в локальном режиме

<img src="pics/schema_local_2.png" width=600/>

## Запуск PySpark

In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))


## SparkContext (sc) - это основной управляющий объект.

In [2]:
from pyspark import SparkContext, SparkConf

config = SparkConf()
config.set("spark.app.name", "Sergey Grishaev Spark RDD app")

sc = SparkContext(conf=config)

## Для получения всех установленных опций конфигурации можно использовать `sc.getConf()`

In [3]:
sc

<SparkContext master=yarn appName=Sergey Grishaev Spark RDD app>

In [4]:
sc.getConf().getAll()

[('spark.history.kerberos.keytab', 'none'),
 ('spark.eventLog.enabled', 'true'),
 ('spark.history.fs.cleaner.maxAge', '7d'),
 ('spark.history.ui.port', '18081'),
 ('spark.driver.extraLibraryPath',
  '/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'),
 ('spark.shuffle.io.serverThreads', '128'),
 ('spark.sql.streaming.streamingQueryListeners', ''),
 ('spark.executor.extraLibraryPath',
  '/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'),
 ('spark.driver.host', 'spark-master-6.newprolab.com'),
 ('spark.jars.repositories', 'https://repos.spark-packages.org/'),
 ('spark.shuffle.file.buffer', '1m'),
 ('spark.sql.hive.convertMetastoreOrc', 'true'),
 ('spark.yarn.dist.files', ''),
 ('spark.sql.autoBroadcastJoinThreshold', '26214400'),
 ('spark.history.fs.cleaner.interval', '1d'),
 ('spark.ui.filters',
  'org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter'),
 ('spark.eventLog.dir', 'hdf

## Основные абстракции Spark
- RDD - Resilient Distributed Dataset - представляет собой распределенную коллекцию данных. RDD обладает свойствами 
    - неизменяемости (операции над RDD приводят к созданию нового RDD) 
    - отказоустойчивости (в случае утери данных на каком либо шаге выполнения данные восстанавливаются из источника) 
    - поддерживает ленивые вычисления (вычисления не производятся если их результат не сохраняется во вне или не используется в других вычислениях)
- DAG - Directed Acyclic Graph - Представляет собой набор вершин и ребер, где вершины представляют собой RDD, а ребра операции над RDD

## Существует два способа создать RDD
- распределить коллекцию объектов с драйвера
- загрузить внешний датасет

## 1. Распределить коллекцию с драйвера

In [10]:
import numpy as np
vocabulary = ("Apache", "Spark", "Hadoop")
numbers = np.random.randint(10, size=10000)
words = np.random.choice(vocabulary, size=10000)
collection = zip(numbers, words)

In [11]:
rdd = sc.parallelize(collection)

In [12]:
rdd.getNumPartitions()

3

In [13]:
rdd.count()

10000

In [14]:
rdd.take(10)

[(2, 'Apache'),
 (4, 'Spark'),
 (7, 'Spark'),
 (5, 'Spark'),
 (4, 'Apache'),
 (6, 'Apache'),
 (8, 'Spark'),
 (6, 'Hadoop'),
 (9, 'Hadoop'),
 (5, 'Apache')]

In [15]:
rdd.getNumPartitions()

3

## 2. Загрузить внешний датасет (датасет загружается из HDFS)

In [17]:
!hdfs dfs -cat /lectures/lecture01/data/ips.txt

192.168.0.1	CHINA
192.168.0.2	CHINA
192.168.0.3	CHINA
192.168.0.4	CHINA
192.168.0.5	CHINA
192.168.0.6	CHINA
192.168.0.7	CHINA
192.168.0.8	CHINA
192.168.0.9	CHINA
192.168.0.10	CHINA
192.168.0.11	CHINA
192.168.0.12	CHINA
192.168.0.13	CHINA
192.168.0.14	CHINA
192.168.0.15	CHINA
192.168.0.16	CHINA
192.168.0.17	CHINA
192.168.0.18	CHINA
192.168.0.19	CHINA
192.168.0.20	CHINA
192.168.0.21	CHINA
192.168.0.22	CHINA
192.168.0.23	CHINA
192.168.0.24	CHINA
192.168.0.25	CHINA
192.168.0.26	CHINA
192.168.0.27	CHINA
192.168.0.28	CHINA
192.168.0.29	CHINA
192.168.0.30	CHINA
192.168.0.31	CHINA
192.168.0.32	CHINA
192.168.0.33	CHINA
192.168.0.34	CHINA
192.168.0.35	CHINA
192.168.0.36	CHINA
192.168.0.37	CHINA
192.168.0.38	CHINA
192.168.0.39	CHINA
192.168.0.40	CHINA
192.168.0.41	CHINA
192.168.0.42	CHINA
192.168.0.43	CHINA
192.168.0.44	CHINA
192.168.0.45	CHINA
192.168.0.46	CHINA
192.168.0.47	CHINA
192.168.0.48	CHINA
192.168.0.49	CHINA
192.168.0.50	CHINA
192.168.0.51	CHINA
192.168.0.52	CHINA
192.168.0.53	CHINA
19

In [18]:
rdd2 = sc.textFile("/lectures/lecture01/data/ips.txt")

In [19]:
rdd2.take(10)

['192.168.0.1\tCHINA',
 '192.168.0.2\tCHINA',
 '192.168.0.3\tCHINA',
 '192.168.0.4\tCHINA',
 '192.168.0.5\tCHINA',
 '192.168.0.6\tCHINA',
 '192.168.0.7\tCHINA',
 '192.168.0.8\tCHINA',
 '192.168.0.9\tCHINA',
 '192.168.0.10\tCHINA']

In [20]:
rdd2.count()

1000

In [21]:
rdd2.getNumPartitions()

2

In [22]:
rdd2.repartition(6).saveAsTextFile("ips_repart_1.txt")

In [23]:
rdd3 = sc.textFile("ips_repart_1.txt")

In [24]:
rdd3.getNumPartitions()

6

In [25]:
!hadoop fs -ls ips_repart_1.txt/

Found 7 items
-rw-r--r--   3 teacher2 teacher2          0 2022-10-07 21:16 ips_repart_1.txt/_SUCCESS
-rw-r--r--   3 teacher2 teacher2       3335 2022-10-07 21:16 ips_repart_1.txt/part-00000
-rw-r--r--   3 teacher2 teacher2       3396 2022-10-07 21:16 ips_repart_1.txt/part-00001
-rw-r--r--   3 teacher2 teacher2       3219 2022-10-07 21:16 ips_repart_1.txt/part-00002
-rw-r--r--   3 teacher2 teacher2       3091 2022-10-07 21:16 ips_repart_1.txt/part-00003
-rw-r--r--   3 teacher2 teacher2       3098 2022-10-07 21:16 ips_repart_1.txt/part-00004
-rw-r--r--   3 teacher2 teacher2       3323 2022-10-07 21:16 ips_repart_1.txt/part-00005


## RDD API состоит из операции двух типов:
- action
- transformation

<img src="pics/rdd_action_transformation.png" width=800/>

### Трансформация преобразовывает RDD в другой RDD и не приводит к вычислению графа

In [26]:
rdd = sc.parallelize(range(1000000))
rdd2 = rdd.filter(lambda x: x % 2)

In [27]:
rdd2.getNumPartitions()

3

### Action заставляет Spark вычислить граф и вернуть результат либо на драйвер, либо во внешнее хранилище

In [28]:
rdd2.count()

500000

In [29]:
rdd2.take(10)

[1, 3, 5, 7, 9, 11, 13, 15, 17, 19]

### Трансформации можно применять одну за другой, никаких вычислений не будет сделано, пока не будет вызван action

In [30]:
rdd = sc.parallelize(range(1000000))

In [31]:
rdd2 = rdd.filter(lambda x: x % 1000).filter(lambda x: x % 7)
rdd2

PythonRDD[24] at RDD at PythonRDD.scala:53

In [32]:
rdd3 = rdd2.map(lambda x: x * 2)
rdd3

PythonRDD[25] at RDD at PythonRDD.scala:53

In [33]:
rdd3.collect()

[2,
 4,
 6,
 8,
 10,
 12,
 16,
 18,
 20,
 22,
 24,
 26,
 30,
 32,
 34,
 36,
 38,
 40,
 44,
 46,
 48,
 50,
 52,
 54,
 58,
 60,
 62,
 64,
 66,
 68,
 72,
 74,
 76,
 78,
 80,
 82,
 86,
 88,
 90,
 92,
 94,
 96,
 100,
 102,
 104,
 106,
 108,
 110,
 114,
 116,
 118,
 120,
 122,
 124,
 128,
 130,
 132,
 134,
 136,
 138,
 142,
 144,
 146,
 148,
 150,
 152,
 156,
 158,
 160,
 162,
 164,
 166,
 170,
 172,
 174,
 176,
 178,
 180,
 184,
 186,
 188,
 190,
 192,
 194,
 198,
 200,
 202,
 204,
 206,
 208,
 212,
 214,
 216,
 218,
 220,
 222,
 226,
 228,
 230,
 232,
 234,
 236,
 240,
 242,
 244,
 246,
 248,
 250,
 254,
 256,
 258,
 260,
 262,
 264,
 268,
 270,
 272,
 274,
 276,
 278,
 282,
 284,
 286,
 288,
 290,
 292,
 296,
 298,
 300,
 302,
 304,
 306,
 310,
 312,
 314,
 316,
 318,
 320,
 324,
 326,
 328,
 330,
 332,
 334,
 338,
 340,
 342,
 344,
 346,
 348,
 352,
 354,
 356,
 358,
 360,
 362,
 366,
 368,
 370,
 372,
 374,
 376,
 380,
 382,
 384,
 386,
 388,
 390,
 394,
 396,
 398,
 400,
 402,
 404,
 4

### `take()` пытается минимизировать число обращений к партициям, поэтому может возвращать смещенные результаты

In [34]:
rdd.take(10)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

### Будьте аккуратны с `collect()`, потому что он загружает все данные из RDD на драйвер. Это может легко привести к Out of Memory exception

In [35]:
rdd.collect()[:20]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

### Если нужно получить небольшое число записей на драйвер и, при этом, сохранить распределение, то лучше сделать выборку

In [36]:
rdd.takeSample(withReplacement=False, num=20, seed=5757)

[347399,
 536788,
 94124,
 169558,
 729189,
 836541,
 973939,
 512558,
 171385,
 123303,
 364321,
 587839,
 139749,
 532891,
 757568,
 969298,
 544812,
 690521,
 175014,
 895109]

## Познакомимся с данными. Будем работать с двумя таблицами

![](pics/data_table1.png)

![](pics/data_table2.png)

### Примеры трансформаций

In [37]:
rdd = sc.textFile("/lectures/lecture01/data/ips.txt")

In [38]:
rdd.getNumPartitions()

2

In [39]:
rdd.take(5)

['192.168.0.1\tCHINA',
 '192.168.0.2\tCHINA',
 '192.168.0.3\tCHINA',
 '192.168.0.4\tCHINA',
 '192.168.0.5\tCHINA']

In [40]:
ips = rdd.map(lambda x: x.split("\t"))

In [41]:
ips.take(5)

[['192.168.0.1', 'CHINA'],
 ['192.168.0.2', 'CHINA'],
 ['192.168.0.3', 'CHINA'],
 ['192.168.0.4', 'CHINA'],
 ['192.168.0.5', 'CHINA']]

In [42]:
ips_filtered = ips.filter(lambda x: x[1] != "CHINA")

In [43]:
ips_filtered.take(5)

[['192.168.0.201', 'RUSSIA'],
 ['192.168.0.202', 'RUSSIA'],
 ['192.168.0.203', 'RUSSIA'],
 ['192.168.0.204', 'RUSSIA'],
 ['192.168.0.205', 'RUSSIA']]

In [44]:
!hdfs dfs -ls /lectures/lecture01/data/log.txt

Found 5 items
-rw-r--r--   3 hdfs hdfs          0 2022-01-06 18:47 /lectures/lecture01/data/log.txt/_SUCCESS
-rw-r--r--   3 hdfs hdfs   12650786 2022-01-06 18:47 /lectures/lecture01/data/log.txt/part-00000
-rw-r--r--   3 hdfs hdfs   12644978 2022-01-06 18:47 /lectures/lecture01/data/log.txt/part-00001
-rw-r--r--   3 hdfs hdfs   12646744 2022-01-06 18:47 /lectures/lecture01/data/log.txt/part-00002
-rw-r--r--   3 hdfs hdfs   12676672 2022-01-06 18:47 /lectures/lecture01/data/log.txt/part-00003


In [45]:
raw_logs = sc.textFile("/lectures/lecture01/data/log.txt")

In [46]:
raw_logs.getNumPartitions()

4

In [47]:
raw_logs.take(5)

['192.168.0.10\tERROR\tWhen production fails in dipsair, whom you gonna call?',
 '192.168.0.39\tINFO\tJust an info message passing by',
 '192.168.0.35\tINFO\tJust an info message passing by',
 '192.168.0.19\tINFO\tJust an info message passing by',
 '192.168.0.23\tERROR\tWhen production fails in dipsair, whom you gonna call?']

In [48]:
logs = raw_logs.map(lambda x: x.split("\t"))

In [49]:
logs.take(5)

[['192.168.0.10',
  'ERROR',
  'When production fails in dipsair, whom you gonna call?'],
 ['192.168.0.39', 'INFO', 'Just an info message passing by'],
 ['192.168.0.35', 'INFO', 'Just an info message passing by'],
 ['192.168.0.19', 'INFO', 'Just an info message passing by'],
 ['192.168.0.23',
  'ERROR',
  'When production fails in dipsair, whom you gonna call?']]

In [50]:
logs.flatMap(lambda x: x[2].split()).take(20)

['When',
 'production',
 'fails',
 'in',
 'dipsair,',
 'whom',
 'you',
 'gonna',
 'call?',
 'Just',
 'an',
 'info',
 'message',
 'passing',
 'by',
 'Just',
 'an',
 'info',
 'message',
 'passing']

In [51]:
words = logs.flatMap(lambda x: x[2].split())

In [52]:
words.take(10)

['When',
 'production',
 'fails',
 'in',
 'dipsair,',
 'whom',
 'you',
 'gonna',
 'call?',
 'Just']

In [55]:
words.groupBy(lambda x: x).getNumPartitions()

4

In [53]:
words.groupBy(lambda x: x).count()

19

## Зачем нужны отдельные трансформации и отдельные action?

![](pics/dag1.png)

![](pics/dag2.png)

### Последовательность трансформаций определяет граф вычислений (DAG - direct acyclic graph). В нем есть партиции и зависимости между партициями. Таким образом Spark имеет всю необходимую информацию для вычилсения графа в любой точке и возможных оптимизаций

![](pics/dag3.png)

### Трансформации бывают *узкими*

![](pics/narrow_transformation.png)

### И *широкими*

![](pics/wide_transformation.png)

### Широкие трансформации разделяют джоб на стейджи. Между стейджами происходит shuffle данных, которого надо избегать. Кстати почему?

## Summary:
    
    - Каждое действие (action) инициирует новое задание (job)
    - Spark анализирует граф RDD и строит план выполнения
    - План выполнения может включать несколько этапов (stages)
    - Каждый этап состоит из набора задач (tasks) - полняется один и тот же код на разных кусочках данных

In [56]:
rdd = sc.textFile("/lectures/lecture01/data/ips.txt")
ips = rdd.map(lambda x: x.split("\t"))
ips_filtered = ips.filter(lambda x: x[1] != "CHINA")
ips_filtered.take(5)

[['192.168.0.201', 'RUSSIA'],
 ['192.168.0.202', 'RUSSIA'],
 ['192.168.0.203', 'RUSSIA'],
 ['192.168.0.204', 'RUSSIA'],
 ['192.168.0.205', 'RUSSIA']]

In [57]:
raw_logs = sc.textFile("/lectures/lecture01/data/log.txt")
logs = raw_logs.map(lambda x: x.split("\t"))
words = logs.flatMap(lambda x: x[2].split())
words.groupBy(lambda x: x).count()
words.groupBy(lambda x: x).count()

19

## Персистентность и кэширование

### RDD вычисляются лениво, когда вызывается action. Часто мы хотим вызвать несколько actions для одного и тоге же RDD. Если мы просто сделаем это, то граф будет полностью перевычисляться каждый раз.

In [58]:
ips.count()

1000

In [59]:
ips.top(10)

[['192.168.3.99', 'USA'],
 ['192.168.3.98', 'USA'],
 ['192.168.3.97', 'USA'],
 ['192.168.3.96', 'USA'],
 ['192.168.3.95', 'USA'],
 ['192.168.3.94', 'USA'],
 ['192.168.3.93', 'USA'],
 ['192.168.3.92', 'USA'],
 ['192.168.3.91', 'USA'],
 ['192.168.3.90', 'USA']]

### Чтобы этого избежать, мы можем закэшировать RDD в памяти. Кэширование произойдет при вызове первого action.

In [60]:
ips_cached = ips.cache()

In [61]:
ips_cached.count()

1000

In [63]:
ips_cached.top(20)

[['192.168.3.99', 'USA'],
 ['192.168.3.98', 'USA'],
 ['192.168.3.97', 'USA'],
 ['192.168.3.96', 'USA'],
 ['192.168.3.95', 'USA'],
 ['192.168.3.94', 'USA'],
 ['192.168.3.93', 'USA'],
 ['192.168.3.92', 'USA'],
 ['192.168.3.91', 'USA'],
 ['192.168.3.90', 'USA'],
 ['192.168.3.9', 'USA'],
 ['192.168.3.89', 'USA'],
 ['192.168.3.88', 'USA'],
 ['192.168.3.87', 'USA'],
 ['192.168.3.86', 'USA'],
 ['192.168.3.85', 'USA'],
 ['192.168.3.84', 'USA'],
 ['192.168.3.83', 'USA'],
 ['192.168.3.82', 'USA'],
 ['192.168.3.81', 'USA']]

### `cache()` сохраняет RDD в памяти. Для большего контроля можно использовать `persist(storage_level)`:
+ MEMORY_ONLY
+ MEMORY_AND_DISK
+ DISK_ONLY
+ MEMORY_ONLY_2
+ MEMORY_AND_DISK_2

## Удаление данных из кэша
 - Автоматическое (LRU cache)

In [64]:
ips_cached.unpersist()

PythonRDD[72] at RDD at PythonRDD.scala:53

### Обработка отказов

- Промежуточные результаты не сохраняются в HDFS
- RDD по-умолчанию не реплицируются
- Для восстановления потерянных данных используется информация о происхождении
- Spark восстанавливает потерянные фрагменты RDD, заново вычисляя их путем применения трансформаций

## PairRDD (ключ-значение)

### PairRDD - это RDD для работы с парами ключ-значение. Spark предполагает, что PairRDD содержит в себе объекты, состящие ровно из двух элементов! PairRDD предоставляют методы группировки, аггрегации и объединения (join) двух RDD

### Пусть есть задача подсчитать распределение кодов ERROR и WARNING в лог-файле

In [65]:
raw_logs.take(5)

['192.168.0.10\tERROR\tWhen production fails in dipsair, whom you gonna call?',
 '192.168.0.39\tINFO\tJust an info message passing by',
 '192.168.0.35\tINFO\tJust an info message passing by',
 '192.168.0.19\tINFO\tJust an info message passing by',
 '192.168.0.23\tERROR\tWhen production fails in dipsair, whom you gonna call?']

In [66]:
(raw_logs.filter(lambda x: "INFO" not in x)
         .map(lambda x: (x.split("\t")[1], 1))\
         .groupByKey()
         .collect())

[('WARNING', <pyspark.resultiterable.ResultIterable at 0x7f50a52a5208>),
 ('ERROR', <pyspark.resultiterable.ResultIterable at 0x7f50a52a5748>)]

In [67]:
(raw_logs.filter(lambda x: "INFO" not in x)
         .map(lambda x: (x.split("\t")[1], 1))\
         .groupByKey()
         .map(lambda x: (x[0], len(x[1])))
         .collect())

[('WARNING', 250004), ('ERROR', 49948)]

### Или немного проще

In [68]:
(raw_logs.filter(lambda x: "INFO" not in x)
         .map(lambda x: (x.split("\t")[1], 1))
         .countByKey()
         .items())

dict_items([('ERROR', 49948), ('WARNING', 250004)])

### Стоит заметить, что `groupByKey()` предполагает перемещение всех записей с одним ключом на один экзекьютор. В случае очень скошенных распределений это может привести к падению экзекьютора с OOM. Поэтому всегда при группировках стоит подумать об использовании `reduceByKey()`.

In [69]:
def plus(x, y):
    return x + y

In [70]:
(raw_logs.filter(lambda x: "INFO" not in x)
         .map(lambda x: (x.split("\t")[1], 1))\
         .reduceByKey(plus)
         .collect())

[('WARNING', 250004), ('ERROR', 49948)]

In [71]:
(raw_logs.filter(lambda x: "INFO" not in x)
         .map(lambda x: (x.split("\t")[1], 1))\
         .reduceByKey(lambda x, y: x + y)
         .collect())

[('ERROR', 49948), ('WARNING', 250004)]

## Join

### Два PairRDD можно объединить по ключу
### Поддерживаются inner join, left outer join, right outer join и full outer join

In [72]:
logs.take(5)

[['192.168.0.10',
  'ERROR',
  'When production fails in dipsair, whom you gonna call?'],
 ['192.168.0.39', 'INFO', 'Just an info message passing by'],
 ['192.168.0.35', 'INFO', 'Just an info message passing by'],
 ['192.168.0.19', 'INFO', 'Just an info message passing by'],
 ['192.168.0.23',
  'ERROR',
  'When production fails in dipsair, whom you gonna call?']]

In [73]:
ips.take(5)

[['192.168.0.1', 'CHINA'],
 ['192.168.0.2', 'CHINA'],
 ['192.168.0.3', 'CHINA'],
 ['192.168.0.4', 'CHINA'],
 ['192.168.0.5', 'CHINA']]

In [74]:
logs.join(ips).take(5)

[('192.168.0.64', ('WARNING', 'CHINA')),
 ('192.168.0.64', ('INFO', 'CHINA')),
 ('192.168.0.64', ('INFO', 'CHINA')),
 ('192.168.0.64', ('INFO', 'CHINA')),
 ('192.168.0.64', ('INFO', 'CHINA'))]

![](pics/Jackie-Chan-WTF.jpg)

### Не стоит забывать, что Spark предполагает, что PairRDD состоит ровно! из двух элементов, поэтому все остальные элементы просто отбрасываются!

In [75]:
def split_logs(line):
    split = line.split("\t")
    return split[0], split[1:]

In [76]:
logs_cached = raw_logs.map(split_logs).cache()

In [77]:
logs_cached.take(5)

[('192.168.0.10',
  ['ERROR', 'When production fails in dipsair, whom you gonna call?']),
 ('192.168.0.39', ['INFO', 'Just an info message passing by']),
 ('192.168.0.35', ['INFO', 'Just an info message passing by']),
 ('192.168.0.19', ['INFO', 'Just an info message passing by']),
 ('192.168.0.23',
  ['ERROR', 'When production fails in dipsair, whom you gonna call?'])]

In [78]:
logs_cached.join(ips).take(5)

[('192.168.0.64', (['WARNING', 'Something bad could happen'], 'CHINA')),
 ('192.168.0.64', (['INFO', 'Just an info message passing by'], 'CHINA')),
 ('192.168.0.64', (['INFO', 'Just an info message passing by'], 'CHINA')),
 ('192.168.0.64', (['INFO', 'Just an info message passing by'], 'CHINA')),
 ('192.168.0.64', (['INFO', 'Just an info message passing by'], 'CHINA'))]

## Управление параллелизмом.

### Вспомним, что атомарным уровнем параллелизма в Spark является партиция. Об этом всегда стоит помнить, когда есть проблемы с производительностью приложения

In [79]:
logs.getNumPartitions()

4

### Метод `repartition()` может быть использован для изменения числа партиций.

In [80]:
logs = logs.repartition(100)

In [81]:
logs.getNumPartitions()

100

### `repartition()` всегда приводит к равномерному перераспределению данных, что ведет к shuffle. Если Вы уменьшаете число партиций, то стоит использовать `coalesce()`, который может избежать shuffle

In [93]:
logs = logs.coalesce(12)

In [94]:
logs.getNumPartitions()

6

In [86]:
logs_1 = logs.coalesce(3)

In [88]:
logs = logs.coalesce(6)

In [89]:
logs.getNumPartitions()

6

### Узнать дефолтный уровень параллелизма можно из конфига. По-умолчанию, при работе с YARN, использукется общее число ядер, выделенных этому SparkContext на всех экзекьюторах, либо 2. Что больше.

In [90]:
print(sc.getConf().get("spark.default.parallelism"))

None


In [95]:
sc.parallelize(range(100000)).getNumPartitions()

3

## Broadcast

### Broadcast-объект - это неизменяемая переменная, которая разделяется между всеми экзекьюторами
### Дистрибуция broadcast-объекта производится быстро и эффективно p2p-протоколом

### Реализуем map-side join с помощью broadcast-объекта

In [96]:
ips_local = dict(ips.collect())

In [97]:
ips_local['192.168.0.10']

'CHINA'

In [98]:
ips_broadcasted = sc.broadcast(ips_local)

In [99]:
ips_broadcasted.value['192.168.0.10']

'CHINA'

In [100]:
logs_cached.take(5)

[('192.168.0.10',
  ['ERROR', 'When production fails in dipsair, whom you gonna call?']),
 ('192.168.0.39', ['INFO', 'Just an info message passing by']),
 ('192.168.0.35', ['INFO', 'Just an info message passing by']),
 ('192.168.0.19', ['INFO', 'Just an info message passing by']),
 ('192.168.0.23',
  ['ERROR', 'When production fails in dipsair, whom you gonna call?'])]

In [101]:
def resolve_ip(row):
    return ips_broadcasted.value[row[0]], row[1:] ## row[0] is the IP address

In [102]:
logs_cached.map(resolve_ip).take(10)

[('CHINA',
  (['ERROR', 'When production fails in dipsair, whom you gonna call?'],)),
 ('CHINA', (['INFO', 'Just an info message passing by'],)),
 ('CHINA', (['INFO', 'Just an info message passing by'],)),
 ('CHINA', (['INFO', 'Just an info message passing by'],)),
 ('CHINA',
  (['ERROR', 'When production fails in dipsair, whom you gonna call?'],)),
 ('CHINA',
  (['ERROR', 'When production fails in dipsair, whom you gonna call?'],)),
 ('CHINA', (['INFO', 'Just an info message passing by'],)),
 ('CHINA', (['WARNING', 'Something bad could happen'],)),
 ('CHINA', (['INFO', 'Just an info message passing by'],)),
 ('CHINA', (['INFO', 'Just an info message passing by'],))]

## Не забудьте погасить SparkContext!

In [103]:
sc.stop()

#### Полезные ссылки
####  https://spark.apache.org/docs/2.4.7/rdd-programming-guide.html

    